# 🚀 Virtual Hands-On Lab: Semantic View Autopilot

Welcome to our **Virtual Hands-On Lab (VHOL)**! In this session, we’ll guide you through an end-to-end, hands-on journey that shows how to go from raw campaign data ➜ to a fully usable **Snowflake Semantic View** ➜ to seamless consumption in **Tableau**.

This lab is designed to be practical, visual, and *fun*—you’ll follow along in a Python notebook and see how each piece fits together in a modern analytics workflow.

---

## 🧭 What You’ll Build

By the end of this lab, you will:

* Load and explore campaign data in Snowflake
* Automatically generate a Semantic View using the **Semantic View Wizard**
* Query Semantic Views using **standard SQL**
* Create a custom **UDF** to generate a Tableau `.tds` file
* Connect Tableau to Snowflake using the generated Semantic View

---

## 🔄 The Big Picture

```
┌──────────────────────┐                 ┌────────────────────────────┐
│  Campaign Data       │                 │ Existing Tableau Workbook  │
│  (Snowflake Tables)  │                 │ (.twb file)                │
└──────────┬───────────┘                 └─────────────┬──────────────┘
           │                                           │
           ▼                                           ▼
┌──────────────────────────────────────────────────────────────────────┐
│ Semantic View Autopilot "Fast-Gen"                                   │
│ (Generate Semantic View from existing analytics)                     │
└──────────┬───────────────────────────────────────────────────────────┘
           │
           ▼
┌──────────────────────┐                 ┌────────────────────────────┐
│ Snowflake Semantic   │                 │ Query History              │
│ View (Metrics + Dims)│                 │ (Real Usage Patterns)      │
└──────────┬───────────┘                 └─────────────┬──────────────┘
           │                                           │
           ▼                                           ▼
┌──────────────────────────────────────────────────────────────────────┐
│ Semantic View Autopilot "Agentic Optimize"                           │
│ (Refine semantics using observed query behavior)                     │
└──────────┬────────────────────────────────────┬──────────────────────┘
           │                                    │
           ▼                                    ▼
┌──────────────────────┐                ┌──────────────────────┐ 
│ Standard SQL Queries │                │ Cortex Analyst       │ 
│ (Human-Friendly!)    │                │ (Text to SQL)        │
└──────────┬───────────┘                └──────────-───────────┘
           │
           ▼
┌──────────────────────┐
│ Tableau .tds File    │
│ (new system function)│
└──────────┬───────────┘
           │
           ▼
┌──────────────────────┐
│ Tableau Dashboard    │
│ Powered by Semantics │
└──────────────────────┘
```

---

## 🧪 Lab Flow (Step by Step)


### 1️⃣ Load the Base Campaign Data
We start by loading and validating the campaign data that will power everything else in the lab. This gives us a clean, trustworthy foundation to build on.


---


### 2️⃣ Generate a Semantic View with Autopilot “Fast-Gen”
Next, we use **Semantic View Autopilot “Fast-Gen”** to generate a Snowflake Semantic View from an **existing Tableau workbook (`.twb`)**. This step shows how business logic can be captured once and reused everywhere.


> ✨ No manual modeling. No duplicated logic. Just semantics.


---


### 3️⃣ Validate & Query the Semantic View
Once the Semantic View exists, we query it using **plain SQL**—no special syntax required. Metrics and dimensions are now first-class citizens.


---


### 4️⃣ Improve Semantics with Autopilot “Agentic Optimize”
Now we take the Semantic View a step further using **Semantic View Autopilot “Agentic Optimize”**.


It uses **Query History** (real usage patterns) to suggest improvements such as:
- Better metric definitions and naming
- Stronger dimension relationships
- More intuitive, human-friendly semantics


The goal is simple: make the model *better the more it’s used*.


---


### 5️⃣ Generate a Tableau `.tds` via the TDS Generator System Function
We then generate a Tableau `.tds` file using our new **TDS Generator system function**. This bridges the gap between Snowflake semantics and Tableau consumption.


---


### 6️⃣ Connect Tableau to the Semantic View
Finally, we use the generated `.tds` file to connect Tableau directly to the Semantic View—unlocking governed, consistent analytics.

---

## 🎯 Why This Matters

- **One semantic layer** → many tools
- **Consistent metrics** across teams
- **Faster time to insight**
- **Less manual modeling**

This lab demonstrates how Snowflake enables *semantic-driven analytics* without sacrificing flexibility or performance.

---

## 🙌 Let’s Get Hands-On

You’re now ready to dive in. Open the notebook, start running cells, and watch your Semantic View come to life.

Happy querying! ❄️📊

```


In [ ]:
--- this script borrows heavily from the Snowflake Intelligence end to end demo here: https://github.com/NickAkincilar/Snowflake_AI_DEMO

--- should take around 1 minute to run completely

-- Switch to accountadmin role to create db, schema, and load data
    USE ROLE ACCOUNTADMIN;

-- Create database and schema
    CREATE OR REPLACE DATABASE SVA_VHOL_DB;
    USE DATABASE SVA_VHOL_DB;

    CREATE SCHEMA IF NOT EXISTS SVA_VHOL_SCHEMA;
    USE SCHEMA SVA_VHOL_SCHEMA;

-- Optional: allow anyone to see the agents in this schema
    GRANT USAGE ON DATABASE SVA_VHOL_DB TO ROLE PUBLIC;
    GRANT USAGE ON SCHEMA SVA_VHOL_DB.SVA_VHOL_SCHEMA TO ROLE PUBLIC;

-- Create file format for CSV files
    CREATE OR REPLACE FILE FORMAT CSV_FORMAT
        TYPE = 'CSV'
        FIELD_DELIMITER = ','
        RECORD_DELIMITER = '\n'
        SKIP_HEADER = 1
        FIELD_OPTIONALLY_ENCLOSED_BY = '"'
        TRIM_SPACE = TRUE
        ERROR_ON_COLUMN_COUNT_MISMATCH = FALSE
        ESCAPE = 'NONE'
        ESCAPE_UNENCLOSED_FIELD = '\134'
        DATE_FORMAT = 'YYYY-MM-DD'
        TIMESTAMP_FORMAT = 'YYYY-MM-DD HH24:MI:SS'
        NULL_IF = ('NULL', 'null', '', 'N/A', 'n/a');

-- Create API Integration for GitHub (public repository access)
    CREATE OR REPLACE API INTEGRATION git_api_integration
        API_PROVIDER = git_https_api
        API_ALLOWED_PREFIXES = ('https://github.com/NickAkincilar/')
        ENABLED = TRUE;
        
-- Create Git repository integration for the public demo repository
    CREATE OR REPLACE GIT REPOSITORY SVA_VHOL_REPO
        API_INTEGRATION = git_api_integration
        ORIGIN = 'https://github.com/NickAkincilar/Snowflake_AI_DEMO.git';

-- Create internal stage for copied data files
    CREATE OR REPLACE STAGE INTERNAL_DATA_STAGE
        FILE_FORMAT = CSV_FORMAT
        COMMENT = 'Internal stage for copied demo data files'
        DIRECTORY = ( ENABLE = TRUE)
        ENCRYPTION = (   TYPE = 'SNOWFLAKE_SSE');

    ALTER GIT REPOSITORY SVA_VHOL_REPO FETCH;

    -- ========================================================================
    -- COPY DATA FROM GIT TO INTERNAL STAGE
    -- ========================================================================

    -- Copy all CSV files from Git repository demo_data folder to internal stage
    COPY FILES
    INTO @INTERNAL_DATA_STAGE/demo_data/
    FROM @SVA_VHOL_REPO/branches/main/demo_data/;


    COPY FILES
    INTO @INTERNAL_DATA_STAGE/unstructured_docs/
    FROM @SVA_VHOL_REPO/branches/main/unstructured_docs/;

    -- Verify files were copied
    LS @INTERNAL_DATA_STAGE;

    ALTER STAGE INTERNAL_DATA_STAGE refresh;

  

    -- ========================================================================
    -- DIMENSION TABLES
    -- ========================================================================

    -- Product Dimension
    CREATE OR REPLACE TABLE product_dim (
        product_key INT PRIMARY KEY,
        product_name VARCHAR(200),
        category_key INT,
        category_name VARCHAR(100),
        vertical VARCHAR(50)
    );

    -- Region Dimension
    CREATE OR REPLACE TABLE region_dim (
        region_key INT PRIMARY KEY,
        region_name VARCHAR(100)
    );

    -- Campaign Dimension (Marketing)
    CREATE OR REPLACE TABLE campaign_dim (
        campaign_key INT PRIMARY KEY,
        campaign_name VARCHAR(300),
        objective VARCHAR(100)
    );

    -- Channel Dimension (Marketing)
    CREATE OR REPLACE TABLE channel_dim (
        channel_key INT PRIMARY KEY,
        channel_name VARCHAR(100)
    );

    -- ========================================================================
    -- FACT TABLE
    -- ========================================================================

    -- Marketing Campaign Fact Table
    CREATE OR REPLACE TABLE marketing_campaign_fact (
        campaign_fact_id INT PRIMARY KEY,
        date DATE ,
        campaign_key INT,
        product_key INT ,
        channel_key INT ,
        region_key INT ,
        spend DECIMAL(10,2) ,
        leads_generated INT ,
        impressions INT 
    );

    -- ========================================================================
    -- LOAD DIMENSION DATA FROM INTERNAL STAGE
    -- ========================================================================

    -- Load Product Dimension
    COPY INTO product_dim
    FROM @INTERNAL_DATA_STAGE/demo_data/product_dim.csv
    FILE_FORMAT = CSV_FORMAT
    ON_ERROR = 'CONTINUE';

    -- Load Region Dimension
    COPY INTO region_dim
    FROM @INTERNAL_DATA_STAGE/demo_data/region_dim.csv
    FILE_FORMAT = CSV_FORMAT
    ON_ERROR = 'CONTINUE';

    -- Load Campaign Dimension
    COPY INTO campaign_dim
    FROM @INTERNAL_DATA_STAGE/demo_data/campaign_dim.csv
    FILE_FORMAT = CSV_FORMAT
    ON_ERROR = 'CONTINUE';

    -- Load Channel Dimension
    COPY INTO channel_dim
    FROM @INTERNAL_DATA_STAGE/demo_data/channel_dim.csv
    FILE_FORMAT = CSV_FORMAT
    ON_ERROR = 'CONTINUE';

    -- ========================================================================
    -- LOAD FACT DATA FROM INTERNAL STAGE
    -- ========================================================================

    -- Load Marketing Campaign Fact
    COPY INTO marketing_campaign_fact
    FROM @INTERNAL_DATA_STAGE/demo_data/marketing_campaign_fact.csv
    FILE_FORMAT = CSV_FORMAT
    ON_ERROR = 'CONTINUE';

    -- ========================================================================
    -- VERIFICATION
    -- ========================================================================

    -- Verify Git integration and file copy
    SHOW GIT REPOSITORIES;
  -- SELECT 'Internal Stage Files' as stage_type, COUNT(*) as file_count FROM (LS @INTERNAL_DATA_STAGE);

    -- Verify data loads
    SELECT 'DIMENSION TABLES' as category, '' as table_name, NULL as row_count
    UNION ALL
    SELECT '', 'product_dim', COUNT(*) FROM product_dim
    UNION ALL
    SELECT '', 'campaign_dim', COUNT(*) FROM campaign_dim
    UNION ALL
    SELECT '', 'channel_dim', COUNT(*) FROM channel_dim
    UNION ALL
    SELECT 'FACT TABLES', '', NULL
    UNION ALL
    SELECT '', 'marketing_campaign_fact', COUNT(*) FROM marketing_campaign_fact;
    

In [ ]:
-- Switch to accountadmin role to create db, schema, and load data
    USE ROLE ACCOUNTADMIN;

-- Create database and schema
    USE DATABASE SVA_VHOL_DB;
    USE SCHEMA SVA_VHOL_SCHEMA;

--- list the tables
    SHOW TABLES;

In [ ]:
--- The semantic view autopilot will look at query history and use this to suggest model improvements and verified queries. If you want to see this in action when you create your semantic view, run this cell.

--- MARKETING SEMANTIC VIEW – ANALYTIC QUERY PLAYBOOK
--- Semantic View: SVA_MARKETING_DASHBOARD_SV

---

--- Query 1: Overall marketing performance by month
--- SQL:

SELECT
DATE_TRUNC('month', mcf.date) AS month,
SUM(mcf.spend) AS total_spend,
SUM(mcf.impressions) AS total_impressions,
SUM(mcf.leads_generated) AS total_leads,
SUM(mcf.spend) / NULLIF(SUM(mcf.leads_generated), 0) AS cost_per_lead,
SUM(mcf.leads_generated) / NULLIF(SUM(mcf.impressions), 0) AS lead_conversion_rate
FROM MARKETING_CAMPAIGN_FACT mcf
GROUP BY 1
ORDER BY 1;

---

--- Query 2: Which channels are most efficient
--- SQL:

SELECT
cd.channel_name,
SUM(mcf.spend) AS total_spend,
SUM(mcf.leads_generated) AS total_leads,
SUM(mcf.spend) / NULLIF(SUM(mcf.leads_generated), 0) AS cost_per_lead
FROM MARKETING_CAMPAIGN_FACT mcf
JOIN CHANNEL_DIM cd ON mcf.channel_key = cd.channel_key
GROUP BY 1
HAVING SUM(mcf.leads_generated) > 0
ORDER BY cost_per_lead ASC, total_leads DESC;

---

--- Query 3: Channel trend – month over month
--- SQL:

SELECT
DATE_TRUNC('month', mcf.date) AS month,
cd.channel_name,
SUM(mcf.spend) AS total_spend,
SUM(mcf.impressions) AS total_impressions,
SUM(mcf.leads_generated) AS total_leads,
SUM(mcf.spend) / NULLIF(SUM(mcf.leads_generated), 0) AS cpl,
SUM(mcf.leads_generated) / NULLIF(SUM(mcf.impressions), 0) AS conv_rate
FROM MARKETING_CAMPAIGN_FACT mcf
JOIN CHANNEL_DIM cd ON mcf.channel_key = cd.channel_key
GROUP BY 1,2
ORDER BY 1,2;

---

--- Query 4: Top campaigns by leads
--- SQL:

SELECT
c.campaign_name,
c.objective,
SUM(mcf.leads_generated) AS total_leads,
SUM(mcf.spend) AS total_spend,
SUM(mcf.impressions) AS total_impressions,
SUM(mcf.spend) / NULLIF(SUM(mcf.leads_generated), 0) AS cpl,
SUM(mcf.leads_generated) / NULLIF(SUM(mcf.impressions), 0) AS conv_rate
FROM MARKETING_CAMPAIGN_FACT mcf
JOIN CAMPAIGN_DIM c ON mcf.campaign_key = c.campaign_key
GROUP BY 1,2
ORDER BY total_leads DESC
LIMIT 20;

---

--- Query 5: Objectives performance
--- SQL:

SELECT
c.objective,
SUM(mcf.spend) AS total_spend,
SUM(mcf.impressions) AS total_impressions,
SUM(mcf.leads_generated) AS total_leads,
SUM(mcf.spend) / NULLIF(SUM(mcf.leads_generated), 0) AS cpl,
SUM(mcf.leads_generated) / NULLIF(SUM(mcf.impressions), 0) AS conv_rate
FROM MARKETING_CAMPAIGN_FACT mcf
JOIN CAMPAIGN_DIM c ON mcf.campaign_key = c.campaign_key
GROUP BY 1
ORDER BY cpl ASC;

---

--- Query 6: Performance by region
--- SQL:

WITH totals AS (
SELECT SUM(spend) AS all_spend, SUM(leads_generated) AS all_leads FROM MARKETING_CAMPAIGN_FACT
)
SELECT
rd.region_name,
SUM(mcf.spend) AS spend,
SUM(mcf.leads_generated) AS leads,
SUM(mcf.spend) / NULLIF(SUM(mcf.leads_generated),0) AS cpl,
SUM(mcf.spend) / NULLIF(t.all_spend,0) AS spend_share,
SUM(mcf.leads_generated) / NULLIF(t.all_leads,0) AS leads_share
FROM MARKETING_CAMPAIGN_FACT mcf
JOIN REGION_DIM rd ON mcf.region_key = rd.region_key
CROSS JOIN totals t
GROUP BY 1, t.all_spend, t.all_leads
ORDER BY leads DESC;

---

--- Query 7: Products with best conversion
--- SQL:

SELECT
pd.product_name,
pd.category_name,
pd.vertical,
SUM(mcf.impressions) AS impressions,
SUM(mcf.leads_generated) AS leads,
SUM(mcf.leads_generated) / NULLIF(SUM(mcf.impressions),0) AS conv_rate
FROM MARKETING_CAMPAIGN_FACT mcf
JOIN PRODUCT_DIM pd ON mcf.product_key = pd.product_key
GROUP BY 1,2,3
HAVING SUM(mcf.impressions) > 0
ORDER BY conv_rate DESC
LIMIT 20;

---

--- Query 8: Category x Channel CPL heatmap
--- SQL:

SELECT
pd.category_name,
cd.channel_name,
SUM(mcf.spend) AS total_spend,
SUM(mcf.leads_generated) AS total_leads,
SUM(mcf.spend) / NULLIF(SUM(mcf.leads_generated),0) AS cpl
FROM MARKETING_CAMPAIGN_FACT mcf
JOIN PRODUCT_DIM pd ON mcf.product_key = pd.product_key
JOIN CHANNEL_DIM cd ON mcf.channel_key = cd.channel_key
GROUP BY 1,2
ORDER BY pd.category_name, cpl ASC;

---

--- Query 9: Vertical by region leads
--- SQL:

SELECT
pd.vertical,
rd.region_name,
SUM(mcf.leads_generated) AS total_leads,
SUM(mcf.spend) AS total_spend,
SUM(mcf.spend) / NULLIF(SUM(mcf.leads_generated),0) AS cpl
FROM MARKETING_CAMPAIGN_FACT mcf
JOIN PRODUCT_DIM pd ON mcf.product_key = pd.product_key
JOIN REGION_DIM rd ON mcf.region_key = rd.region_key
GROUP BY 1,2
ORDER BY total_leads DESC;

---

--- Query 10: Worst CPL segments
--- SQL:

SELECT
c.campaign_name,
cd.channel_name,
SUM(mcf.leads_generated) AS leads,
SUM(mcf.spend) AS spend,
SUM(mcf.spend) / NULLIF(SUM(mcf.leads_generated),0) AS cpl
FROM MARKETING_CAMPAIGN_FACT mcf
JOIN CAMPAIGN_DIM c ON mcf.campaign_key = c.campaign_key
JOIN CHANNEL_DIM cd ON mcf.channel_key = cd.channel_key
GROUP BY 1,2
HAVING SUM(mcf.leads_generated) >= 50
ORDER BY cpl DESC
LIMIT 10;

--- Query 11: Budget allocation by objective and channel
--- SQL:

WITH obj_totals AS (
SELECT
c.objective,
SUM(mcf.spend) AS objective_spend
FROM MARKETING_CAMPAIGN_FACT mcf
JOIN CAMPAIGN_DIM c
ON mcf.campaign_key = c.campaign_key
GROUP BY 1
)
SELECT
c.objective,
cd.channel_name,
SUM(mcf.spend) AS spend,
SUM(mcf.spend) / NULLIF(ot.objective_spend, 0) AS spend_share_within_objective
FROM MARKETING_CAMPAIGN_FACT mcf
JOIN CAMPAIGN_DIM c
ON mcf.campaign_key = c.campaign_key
JOIN CHANNEL_DIM cd
ON mcf.channel_key = cd.channel_key
JOIN obj_totals ot
ON c.objective = ot.objective
GROUP BY 1, 2, ot.objective_spend
ORDER BY c.objective, spend_share_within_objective DESC;

--- Query 12: Facebook vs non-Facebook efficiency over time
--- SQL:

SELECT
DATE_TRUNC('month', mcf.date) AS month,
SUM(CASE WHEN cd.channel_name = 'Facebook' THEN mcf.spend ELSE 0 END) AS facebook_spend,
SUM(CASE WHEN cd.channel_name <> 'Facebook' THEN mcf.spend ELSE 0 END) AS non_facebook_spend,
SUM(CASE WHEN cd.channel_name = 'Facebook' THEN mcf.leads_generated ELSE 0 END) AS facebook_leads,
SUM(CASE WHEN cd.channel_name <> 'Facebook' THEN mcf.leads_generated ELSE 0 END) AS non_facebook_leads,
(SUM(CASE WHEN cd.channel_name = 'Facebook' THEN mcf.spend ELSE 0 END)
/ NULLIF(SUM(CASE WHEN cd.channel_name = 'Facebook' THEN mcf.leads_generated ELSE 0 END), 0)
) AS facebook_cpl,
(SUM(CASE WHEN cd.channel_name <> 'Facebook' THEN mcf.spend ELSE 0 END)
/ NULLIF(SUM(CASE WHEN cd.channel_name <> 'Facebook' THEN mcf.leads_generated ELSE 0 END), 0)
) AS non_facebook_cpl
FROM MARKETING_CAMPAIGN_FACT mcf
JOIN CHANNEL_DIM cd
ON mcf.channel_key = cd.channel_key
GROUP BY 1
ORDER BY 1;

--- Query 13: Campaign flighting (first/last active date, duration, totals)
--- SQL:

SELECT
c.campaign_name,
c.objective,
MIN(mcf.date) AS first_active_date,
MAX(mcf.date) AS last_active_date,
DATEDIFF('day', MIN(mcf.date), MAX(mcf.date)) + 1 AS active_days,
SUM(mcf.spend) AS total_spend,
SUM(mcf.leads_generated) AS total_leads,
SUM(mcf.impressions) AS total_impressions,
SUM(mcf.spend) / NULLIF(SUM(mcf.leads_generated), 0) AS cpl,
SUM(mcf.leads_generated) / NULLIF(SUM(mcf.impressions), 0) AS conv_rate
FROM MARKETING_CAMPAIGN_FACT mcf
JOIN CAMPAIGN_DIM c
ON mcf.campaign_key = c.campaign_key
GROUP BY 1, 2
ORDER BY last_active_date DESC, total_spend DESC;

--- Query 14: Week-over-week growth (spend and leads with WoW % change)
--- SQL:

WITH weekly AS (
SELECT
DATE_TRUNC('week', date) AS week,
SUM(spend) AS spend,
SUM(leads_generated) AS leads
FROM MARKETING_CAMPAIGN_FACT
GROUP BY 1
)
SELECT
week,
spend,
leads,
(spend - LAG(spend) OVER (ORDER BY week))
/ NULLIF(LAG(spend) OVER (ORDER BY week), 0) AS spend_wow_pct,
(leads - LAG(leads) OVER (ORDER BY week))
/ NULLIF(LAG(leads) OVER (ORDER BY week), 0) AS leads_wow_pct
FROM weekly
ORDER BY week;

--- Query 15: Best ‘scaled’ segments (lowest CPL with high lead volume, region x channel)
--- SQL:

SELECT
rd.region_name,
cd.channel_name,
SUM(mcf.leads_generated) AS leads,
SUM(mcf.spend) AS spend,
SUM(mcf.spend) / NULLIF(SUM(mcf.leads_generated), 0) AS cpl
FROM MARKETING_CAMPAIGN_FACT mcf
JOIN REGION_DIM rd
ON mcf.region_key = rd.region_key
JOIN CHANNEL_DIM cd
ON mcf.channel_key = cd.channel_key
GROUP BY 1, 2
HAVING SUM(mcf.leads_generated) >= 100
ORDER BY cpl ASC, leads DESC
LIMIT 20;

--- Query 16: Objective mix by region (what are we running where?)
--- SQL:

SELECT
rd.region_name,
c.objective,
SUM(mcf.spend) AS spend,
SUM(mcf.leads_generated) AS leads,
SUM(mcf.impressions) AS impressions,
SUM(mcf.spend) / NULLIF(SUM(mcf.leads_generated), 0) AS cpl,
SUM(mcf.leads_generated) / NULLIF(SUM(mcf.impressions), 0) AS conv_rate
FROM MARKETING_CAMPAIGN_FACT mcf
JOIN REGION_DIM rd
ON mcf.region_key = rd.region_key
JOIN CAMPAIGN_DIM c
ON mcf.campaign_key = c.campaign_key
GROUP BY 1, 2
ORDER BY rd.region_name, spend DESC;

--- Query 17: Pareto view (cumulative leads vs cumulative spend by campaign)
--- SQL:

WITH by_campaign AS (
SELECT
c.campaign_name,
SUM(mcf.leads_generated) AS leads,
SUM(mcf.spend) AS spend
FROM MARKETING_CAMPAIGN_FACT mcf
JOIN CAMPAIGN_DIM c
ON mcf.campaign_key = c.campaign_key
GROUP BY 1
),
ranked AS (
SELECT
*,
SUM(leads) OVER () AS total_leads,
SUM(spend) OVER () AS total_spend,
SUM(leads) OVER (ORDER BY leads DESC) AS cum_leads,
SUM(spend) OVER (ORDER BY leads DESC) AS cum_spend
FROM by_campaign
)
SELECT
campaign_name,
leads,
spend,
cum_leads / NULLIF(total_leads, 0) AS cumulative_leads_share,
cum_spend / NULLIF(total_spend, 0) AS cumulative_spend_share
FROM ranked
ORDER BY leads DESC;

--- Query 18: Anomaly detection-lite (days with unusually high CPL via z-score)
--- SQL:

WITH daily AS (
SELECT
date,
SUM(spend) AS spend,
SUM(leads_generated) AS leads,
SUM(spend) / NULLIF(SUM(leads_generated), 0) AS cpl
FROM MARKETING_CAMPAIGN_FACT
GROUP BY 1
),
stats AS (
SELECT
AVG(cpl) AS avg_cpl,
STDDEV_SAMP(cpl) AS std_cpl
FROM daily
WHERE cpl IS NOT NULL
),
scored AS (
SELECT
d.date,
d.spend,
d.leads,
d.cpl,
(d.cpl - s.avg_cpl) / NULLIF(s.std_cpl, 0) AS cpl_zscore
FROM daily d
CROSS JOIN stats s
WHERE d.cpl IS NOT NULL
)
SELECT
*
FROM scored
WHERE ABS(cpl_zscore) >= 2
ORDER BY ABS(cpl_zscore) DESC, date DESC;

--- Query 19: Diminishing returns curve (CPL by daily spend bucket)
--- SQL:

WITH daily AS (
SELECT
date,
SUM(spend) AS spend,
SUM(leads_generated) AS leads
FROM MARKETING_CAMPAIGN_FACT
GROUP BY 1
),
bucketed AS (
SELECT
*,
WIDTH_BUCKET(spend, 0, (SELECT MAX(spend) FROM daily), 10) AS spend_bucket
FROM daily
)
SELECT
spend_bucket,
MIN(spend) AS min_spend_in_bucket,
MAX(spend) AS max_spend_in_bucket,
SUM(spend) AS total_spend,
SUM(leads) AS total_leads,
SUM(spend) / NULLIF(SUM(leads), 0) AS cpl
FROM bucketed
GROUP BY 1
ORDER BY 1;

--- Query 20: Efficiency frontier (top 5 campaigns per channel by lowest CPL, min lead threshold)
--- SQL:

WITH campaign_channel AS (
SELECT
cd.channel_name,
c.campaign_name,
SUM(mcf.spend) AS spend,
SUM(mcf.leads_generated) AS leads,
SUM(mcf.spend) / NULLIF(SUM(mcf.leads_generated), 0) AS cpl
FROM MARKETING_CAMPAIGN_FACT mcf
JOIN CHANNEL_DIM cd
ON mcf.channel_key = cd.channel_key
JOIN CAMPAIGN_DIM c
ON mcf.campaign_key = c.campaign_key
GROUP BY 1, 2
HAVING SUM(mcf.leads_generated) >= 50
),
ranked AS (
SELECT
*,
ROW_NUMBER() OVER (
PARTITION BY channel_name
ORDER BY cpl ASC NULLS LAST, leads DESC
) AS rn
FROM campaign_channel
)
SELECT
channel_name,
campaign_name,
spend,
leads,
cpl
FROM ranked
WHERE rn <= 5
ORDER BY channel_name, rn;

### Use the Semantic View Wizard to Build a Semantic View from a Tableau Dashbard

- Navigate to the Semantic View wizard by going to **"AI & ML > Analyst"** from the left side menu in Snowflake, pick the `SVA_VHOL_DB.SVA_VHOL_SCHEMA` from the drop down, and select **"Create New Semantic View"**. *Note: be sure to use the `ACCOUNTADMIN` role or a role that has ownership rights on `SVA_VHOL_DB.SVA_VHOL_SCHEMA.`*
- Name your semantic view `SVA_MARKETING_SV` and click **"Next"**. In the following screen, select "Tableu Files" as your context. Pick `SVA_VHOL_SCHEMA.INTERNAL_DATA_STAGE` and navigate to the `/unstructured_docs/BI_dashboards` folder (when in `/unstructured_docs/` folder you may need to scroll down to find `BI_dashboards` sub-folder. Select the `CampaignMetricsDash.twb` file and click "Create and Save".
- On the next screen you will now see a "first pass" version of a Snowflake Semantic view that is based on the imported `.twb` file.  Test it out by going to the **"Playground"** tab in the right-hand panel and asking: *"Show me the top 10 most expensive products based on cost per lead"*.
- Congratulations! You just went from "zero-to-semantic-view" in 1 minute!


### Use Semantic View "Standard SQL" to Query Semantic Views, Access Using Tableau

Semantic View [Standard SQL](https://docs.snowflake.com/en/user-guide/views-semantic/querying#specifying-the-name-of-the-semantic-view-in-the-from-clause) in Snowflake allows you to right ANSI-style SQL queries against semantic views. In this section we will:
- Write and run a Standard SQL query against the `SVA_MARKETING_SV` semantic view
- Generate a Tableau Data Source (`.tds`) file for the semantic view
- Query the semantic view from Tableau



In [ ]:
-- Switch to accountadmin role to create db, schema, and load data
    USE ROLE ACCOUNTADMIN;

-- Create database and schema
    USE DATABASE SVA_VHOL_DB;
    USE SCHEMA SVA_VHOL_SCHEMA;

-- Standard SQL query

SELECT
   -- dimensions and facts can be selected directly
   product_name,
   --- metrics must be aggregated using AGG, MIN, MAX, or ANYVALUE
   AGG(cost_per_lead) as total_cost_per_lead
FROM
   --- directly reference the semantic view name
   SVA_MARKETING_SV
    --- you must include a GROUP BY if your SELECT statement includes any metrics
    GROUP BY ALL
ORDER BY total_cost_per_lead DESC
LIMIT 10;

### Generate and download the .tds file

Snowflake now has a system function that automatically generates a Tableau Data Source (`.tds`) file that points directly to the relevant Snowflake Semantic View with a pre-configured connection, folders for organization, and well-mapped metric and dimensions.

You can also try the function yourself with the statement below:

```select SYSTEM$EXPORT_TDS_FROM_SEMANTIC_VIEW('my_test_sv');```

Run this cell and click **"Generate TDS"**, then open the `.tds` file using Tableau!

In [ ]:
# Clean TDS Generator for Snowflake Notebook
# Simple, minimal interface for developers

import streamlit as st
import zipfile
import io
from snowflake.snowpark.context import get_active_session

def generate_and_download_tds():
    """Clean interface for TDS generation and download"""
    
    st.title("TDS Generator")
    st.write("Generate Tableau Data Source files from Snowflake Semantic Views")
    
    # Input section
    semantic_view = st.text_input(
        "Semantic View Name",
        value="SVA_VHOL_DB.SVA_VHOL_SCHEMA.SVA_MARKETING_SV",
        help="Enter the fully qualified semantic view name"
    )
    
    if st.button("Generate TDS", type="primary"):
        if not semantic_view:
            st.error("Please enter a semantic view name")
            return
            
        try:
            # Get Snowflake session
            session = get_active_session()
            
            # Call the stored procedure
            with st.spinner("Generating TDS file..."):
                result = session.sql(f"select SYSTEM$EXPORT_TDS_FROM_SEMANTIC_VIEW('{semantic_view}');").collect()
                tds_content = result[0][0]
            
            # Check for errors
            if tds_content.startswith("<!-- Error"):
                st.error(f"Generation failed: {tds_content}")
                return
            
            # Success - show download
            st.success("TDS file generated successfully")
            
            # Create filename
            view_name = semantic_view.split('.')[-1] if '.' in semantic_view else semantic_view
            filename = f"{view_name}_Semantic_View.tds"
            
            # Download button
            st.download_button(
                label="Download TDS",
                data=tds_content,
                file_name=filename,
                mime="application/xml"
            )
            
            # Show file info
            st.info(f"File: {filename} ({len(tds_content):,} bytes)")
            
        except Exception as e:
            st.error(f"Error: {str(e)}")

def batch_generate_tds():
    """Generate multiple TDS files at once"""
    
    st.title("Batch TDS Generator")
    
    # Text area for multiple semantic views
    semantic_views = st.text_area(
        "Semantic View Names (one per line)",
        placeholder="DATABASE.SCHEMA.VIEW1\nDATABASE.SCHEMA.VIEW2\nDATABASE.SCHEMA.VIEW3",
        height=150
    )
    
    if st.button("Generate All TDS Files", type="primary"):
        if not semantic_views.strip():
            st.error("Please enter at least one semantic view name")
            return
            
        view_list = [v.strip() for v in semantic_views.split('\n') if v.strip()]
        
        try:
            session = get_active_session()
            zip_buffer = io.BytesIO()
            
            with zipfile.ZipFile(zip_buffer, 'w', zipfile.ZIP_DEFLATED) as zip_file:
                progress_bar = st.progress(0)
                
                for i, semantic_view in enumerate(view_list):
                    st.write(f"Processing: {semantic_view}")
                    
                    # Generate TDS
                    result = session.sql(f"select SYSTEM$EXPORT_TDS_FROM_SEMANTIC_VIEW('{semantic_view}');").collect()
                    tds_content = result[0][0]
                    
                    if not tds_content.startswith("<!-- Error"):
                        # Add to zip
                        view_name = semantic_view.split('.')[-1] if '.' in semantic_view else semantic_view
                        filename = f"{view_name}_Semantic_View.tds"
                        zip_file.writestr(filename, tds_content)
                        st.success(f"✓ {filename}")
                    else:
                        st.error(f"✗ Failed: {semantic_view}")
                    
                    progress_bar.progress((i + 1) / len(view_list))
            
            # Download zip file
            zip_buffer.seek(0)
            st.download_button(
                label="Download All TDS Files (ZIP)",
                data=zip_buffer.getvalue(),
                file_name="semantic_view_tds_files.zip",
                mime="application/zip"
            )
            
        except Exception as e:
            st.error(f"Error: {str(e)}")

# Main interface
def main():
    """Main application"""
    
    # Sidebar for mode selection
    mode = st.sidebar.radio(
        "Mode",
        ["Single TDS", "Batch TDS"]
    )
    
    if mode == "Single TDS":
        generate_and_download_tds()
    else:
        batch_generate_tds()

if __name__ == "__main__":
    main()
